In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_27c_test2.pth

In [1]:
!lscpu

Architecture:                    aarch64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              1
Core(s) per socket:              4
Socket(s):                       1
Vendor ID:                       ARM
Model:                           3
Model name:                      Cortex-A72
Stepping:                        r0p3
CPU max MHz:                     1500.0000
CPU min MHz:                     600.0000
BogoMIPS:                        108.00
Vulnerability Itlb multihit:     Not affected
Vulnerability L1tf:              Not affected
Vulnerability Mds:               Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Vulnerable
Vulnerability Spectre v1:        Mitigation; __user pointer sanitization
Vulnerability Spectre v2:        Vulnerable
Vulnerability Srbds:             Not affected
Vulnerability Tsx

In [2]:
import io
import time
import torch
import numpy as np
import PIL
import torchaudio
import matplotlib.pyplot as plt
from einops import rearrange
from IPython.display import Audio
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc
class Config: pass

In [3]:
checkpoint = torch.load("Stereo_Li_27c_test2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dan/.local/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
MUSDB = load_dataset("danjacobellis/musdb_segments_val", split='validation')

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
def walloc_compress_cpu(sample):
    with torch.no_grad():
        x, fs = torchaudio.load(sample['audio_mix']['bytes'],normalize=False)
        x = x.to(torch.float)
        x = x - x.mean()
        max_abs = x.abs().max()
        x = x / (max_abs + 1e-8)
        x = x/2
        
        L = x.shape[-1]
        x_padded = pad(x.unsqueeze(0), 2**16).to(device)
        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        ℓ = Y.shape[-1]
        Y = pad(Y,256)
        Y = rearrange(Y, 'b c (w h) -> b c w h', h=256).to("cpu")
        webp = walloc.latent_to_pil(Y,codec.latent_bits,3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
    
        t0 = time.time()
        Y = walloc.pil_to_latent([PIL.Image.open(buff)], codec.latent_dim, codec.latent_bits, 3).to(device)
        X_hat = codec.decoder(rearrange(Y.to(device), 'b c h w -> b c (h w)')[:,:,:ℓ])
        x_hat = codec.wavelet_synthesis(X_hat,codec.J)
        x_hat = codec.post(x_hat)
        x_hat = codec.clamp(x_hat[0,:,:L])
        decode_time = time.time() - t0
        
    return {
        'cpu_encode_time': encode_time,
        'cpu_decode_time': decode_time,
        'shape': x.shape,
    }

In [7]:
device = "cpu"
codec = codec.to(device)
cpu = MUSDB.select(range(8)).map(walloc_compress_cpu, writer_batch_size=8)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [8]:
upload = cpu.remove_columns(['audio_mix', 'path_vocal', 'path_bass', 'path_drums', 'path_other'])

In [9]:
enc_samp_per_sec = np.prod(upload[0]['shape'])/np.array(upload['cpu_encode_time'])
print(f"Encode: {np.mean(enc_samp_per_sec)/1e6 : .5g} megasamples per second")

Encode:  2.708 megasamples per second


In [10]:
dec_sec_per_samp = np.array(upload['cpu_decode_time'])/np.prod(upload[0]['shape'])
print(f"Decode: {1e6*np.mean(dec_sec_per_samp) : .5g} seconds per megasample")

Decode:  5.6591 seconds per megasample


In [11]:
upload.push_to_hub("danjacobellis/MUSDB_walloc_20x_raspi",split='validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/MUSDB_walloc_20x_raspi/commit/6409553d099de697ee5f61d6c85fad1ab2457988', commit_message='Upload dataset', commit_description='', oid='6409553d099de697ee5f61d6c85fad1ab2457988', pr_url=None, pr_revision=None, pr_num=None)